In [1]:
import torch

print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("사용 가능한 GPU 개수:", torch.cuda.device_count())
print("사용 중인 GPU 이름:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "GPU 없음")


CUDA 사용 가능 여부: True
사용 가능한 GPU 개수: 1
사용 중인 GPU 이름: NVIDIA GeForce RTX 4060 Laptop GPU


 label-studio 

In [ ]:
``from ultralytics import YOLO
import torch
# 베이스 모델 로드
model = YOLO("yolo11n.pt").to("cuda")

# 모델 학습
model.train(data='datasets/hair/hair.yaml',
            epochs=10,
            project='hair',
            name='hair_model',
            device='cuda',
            amp = False)``

Ultralytics 8.3.82  Python-3.11.7 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=datasets/hair/hair.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=hair, name=hair_model2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

train: Scanning C:\Users\goesy\PycharmProjects\hair_vision_project\datasets\hair\labels\train.cache... 550 images, 1 backgrounds, 0 corrupt: 100%|██████████| 550/550 [00:00<?, ?it/s]
val: Scanning C:\Users\goesy\PycharmProjects\hair_vision_project\datasets\hair\labels\validation.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to hair\hair_model2\labels.jpg... 


In [ ]:
import torch
from pathlib import Path 
import matplotlib.pyplot as plt
from PIL import Image
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("hair/hair_model8/weights/best.pt")

# 평가 데이터 경로
root = Path("datasets/hair/images/test")
output_dir = Path("datasets/hair/cropped")  # 크롭된 이미지 저장 폴더
output_dir.mkdir(parents=True, exist_ok=True)  # 폴더가 없으면 생성

plt.figure(figsize=(20, 20))
i = 0

for path in root.glob("*.png"):
    # 모델 예측
    results = model(path)
    im_array = results[0].plot()  # 시각화용 이미지 배열
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL 이미지 변환

    # 원본 이미지 로드
    original_image = Image.open(path)

    # 예측된 객체별로 바운딩 박스 크롭 후 저장
    boxes = results[0].boxes.xyxy.cpu().numpy()  # 감지된 객체의 바운딩 박스 좌표 (x1, y1, x2, y2)

    for j, (x1, y1, x2, y2) in enumerate(boxes):
        cropped = original_image.crop((x1, y1, x2, y2))  # 객체 크롭
        cropped_path = output_dir / f"{path.stem}_object{j}.png"  # 저장 경로
        cropped.save(cropped_path)  # 크롭된 이미지 저장

    # 시각화
    i += 1
    ax = plt.subplot(3, 4, i)
    ax.imshow(im)
    ax.axis("off")  # 축 제거

plt.tight_layout()
plt.show()
